# Modelling

In [1]:
import pandas as pd

# Load the data
df = pd.read_csv('Data/clean_data.csv')

df

,sentiment,raw_comm,nouns_comm
0,neutral,Technopolis plans to develop in stages an area...,Technopolis stages area meters order companies...
1,negative,The international electronic industry company ...,industry company tens employees facility layof...
2,positive,With the new production plant the company woul...,production plant company capacity increase dem...
3,positive,According to the company 's updated strategy f...,company strategy years term sales growth range...
4,positive,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is ag...,FINANCING growth strategy circuit boards PCBs
...,...,...,...
4840,negative,LONDON MarketWatch -- Share prices ended lower...,Share prices rebound bank stocks weakness
4841,neutral,Rinkuskiai 's beer sales fell by 6.5 per cent ...,beer sales per cent litres beer sales cent litres
4842,negative,Operating profit fell to EUR 35.4 mn from EUR ...,profit mn mn vessel sales gain mn
4843,negative,Net sales of the Paper segment decreased to EU...,sales segment mn quarter mn quarter profit ite...


### Split the data

In [2]:
from sklearn.model_selection import train_test_split

# Split the data into training and test sets
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['sentiment'], random_state=42)

# Display the class distribution in the training set before downsampling
print("Training set class distribution before downsampling:")
print(train_df['sentiment'].value_counts())

# Display the class distribution in the test set
print("Test set class distribution:")
print(test_df['sentiment'].value_counts())

Training set class distribution before downsampling:
neutral     2302
positive    1091
negative     483
Name: sentiment, dtype: int64
Test set class distribution:
neutral     576
positive    272
negative    121
Name: sentiment, dtype: int64


### Up sampling

In [3]:
# Determine the size of the largest class in the training set
max_class_size = train_df['sentiment'].value_counts().max()

# Upsample each sentiment class in the training set
train_positive = train_df[train_df['sentiment'] == 'positive'].sample(max_class_size, replace=True, random_state=42)
train_negative = train_df[train_df['sentiment'] == 'negative'].sample(max_class_size, replace=True, random_state=42)
train_neutral = train_df[train_df['sentiment'] == 'neutral'].sample(max_class_size, replace=True, random_state=42)

# Combine the downsampled dataframes
train_df_downsampled = pd.concat([train_positive, train_negative, train_neutral])

# Shuffle the combined dataframe to mix the classes
train_df_downsampled = train_df_downsampled.sample(frac=1, random_state=42).reset_index(drop=True)

# Display the class distribution after downsampling in the training set
print("Training set class distribution after downsampling:")
print(train_df_downsampled['sentiment'].value_counts())


Training set class distribution after downsampling:
negative    2302
positive    2302
neutral     2302
Name: sentiment, dtype: int64


### TF-IDF Vectorization

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Fill NaN values in 'nouns_comm' and 'raw_comm' columns with an empty string
train_df_downsampled['nouns_comm'] = train_df_downsampled['nouns_comm'].fillna('')
train_df_downsampled['raw_comm'] = train_df_downsampled['raw_comm'].fillna('')
test_df['nouns_comm'] = test_df['nouns_comm'].fillna('')
test_df['raw_comm'] = test_df['raw_comm'].fillna('')

# TF-IDF Vectorization for 'nouns_comm' and 'raw_comm'
tfidf_vectorizer_nouns = TfidfVectorizer()
tfidf_vectorizer_raw = TfidfVectorizer()

X_train_nouns = tfidf_vectorizer_nouns.fit_transform(train_df_downsampled['nouns_comm'])
X_test_nouns = tfidf_vectorizer_nouns.transform(test_df['nouns_comm'])

X_train_raw = tfidf_vectorizer_raw.fit_transform(train_df_downsampled['raw_comm'])
X_test_raw = tfidf_vectorizer_raw.transform(test_df['raw_comm'])

y_train = train_df_downsampled['sentiment']
y_test = test_df['sentiment']


## Naive Bayes Model

### Train the model

In [5]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score

# Train Naive Bayes models for both 'nouns_comm' and 'raw_comm'
nb_nouns = MultinomialNB()
nb_raw = MultinomialNB()   

# Fit the models
nb_nouns.fit(X_train_nouns, y_train)
nb_raw.fit(X_train_raw, y_train)

# Predict on the training set
y_train_pred_nouns = nb_nouns.predict(X_train_nouns)
y_train_pred_raw = nb_raw.predict(X_train_raw)

# Predict on the test set
y_test_pred_nouns = nb_nouns.predict(X_test_nouns)
y_test_pred_raw = nb_raw.predict(X_test_raw)


### Compute models' accuracy

In [14]:
# Evaluate the Naive Bayes model on the test set ('nouns_comm')
print("\nTest Set - Naive Bayes Model on 'nouns_comm'")
print("Accuracy:", accuracy_score(y_test, y_test_pred_nouns))
print("Classification Report:\n", classification_report(y_test, y_test_pred_nouns))

# Evaluate the Naive Bayes model on the test set ('raw_comm')
print("\nTest Set - Naive Bayes Model on 'raw_comm'")
print("Accuracy:", accuracy_score(y_test, y_test_pred_raw))
print("Classification Report:\n", classification_report(y_test, y_test_pred_raw))



Test Set - Naive Bayes Model on 'nouns_comm'
Accuracy: 0.5830753353973168
Classification Report:
               precision    recall  f1-score   support

    negative       0.33      0.65      0.44       121
     neutral       0.79      0.64      0.71       576
    positive       0.45      0.43      0.44       272

    accuracy                           0.58       969
   macro avg       0.52      0.57      0.53       969
weighted avg       0.64      0.58      0.60       969


Test Set - Naive Bayes Model on 'raw_comm'
Accuracy: 0.6800825593395253
Classification Report:
               precision    recall  f1-score   support

    negative       0.44      0.80      0.57       121
     neutral       0.83      0.73      0.78       576
    positive       0.57      0.51      0.54       272

    accuracy                           0.68       969
   macro avg       0.62      0.68      0.63       969
weighted avg       0.71      0.68      0.69       969



### Add predictions in the test_df

In [20]:
test_df['NB_pred_noun'] = y_test_pred_nouns
test_df['NB_pred_raw'] = y_test_pred_raw

test_df

,sentiment,raw_comm,nouns_comm,NB_pred_noun,NB_pred_raw
2783,neutral,"ALEXANDRIA , Va. , Dec. 17 -- Reijo Salovaara ...",veneer cutter,neutral,neutral
1566,positive,Nokia will continue to invest in future develo...,development benefit users,neutral,neutral
478,neutral,"Baltic Pearl CJSC , a subsidiary of Shanghai F...",subsidiary project meters space hospitals scho...,neutral,neutral
911,positive,This combined with foreign investments creates...,investments opportunities,neutral,neutral
1294,neutral,"Of course , you 'll have direct access to Noki...",access store lots fun media,neutral,neutral
...,...,...,...,...,...
4535,positive,Key shareholders of Finnish IT services provid...,shareholders services provider offer shop life...,neutral,neutral
141,positive,Finnish Bank of +_land reports its operating p...,land operating profit mn quarter mn period,negative,negative
945,positive,"EBITDA for the year 2009 improved to EUR484m ,...",EBITDA year,positive,positive
1523,positive,Finnish retailer Stockmann has won approval fr...,retailer approval board tender offer aim compa...,neutral,positive


## FNN

### FNN with nouns comm

#### Create the FNN

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

def create_fnn_model(input_dim, layers=1, nodes=32, learning_rate=0.001, dropout_rate=0.5):
    """
    Creates a feedforward neural network model.
    
    Args:
        input_dim (int): Number of input features.
        layers (int): Number of hidden layers.
        nodes (int): Number of nodes per layer.
        learning_rate (float): Learning rate for the optimizer.
        dropout_rate (float): Dropout rate to prevent overfitting.

    Returns:
        model: Compiled Keras model.
    """
    model = Sequential()
    model.add(Dense(nodes, input_dim=input_dim, activation='relu'))
    
    # Add hidden layers
    for _ in range(layers - 1):
        model.add(Dense(nodes, activation='relu'))
        model.add(Dropout(dropout_rate))
    
    # Output layer with 3 nodes for the 3 classes
    model.add(Dense(3, activation='softmax'))
    
    # Compile the model
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model


2024-08-17 11:58:13.487816: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


#### Create the Grid Search

In [13]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV

# Function to create the KerasClassifier model for grid search
def build_keras_classifier(input_dim, layers, nodes, learning_rate):
    return KerasClassifier(model=create_fnn_model, input_dim=input_dim, layers=layers, 
                           nodes=nodes, learning_rate=learning_rate, verbose=1)

# Define hyperparameters for grid search
param_grid = {
    'layers': [1, 2, 3],
    'nodes': [32, 64, 128],
    'learning_rate': [0.001, 0.01, 0.1],
    'epochs': [20, 30],
    'batch_size': [32, 64]
}

# Assuming X_train_nouns and y_train are defined
input_dim = X_train_nouns.shape[1]
model = build_keras_classifier(input_dim=input_dim, layers=1, nodes=32, learning_rate=0.001)

# Perform Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train_nouns, y_train)


144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.3240 - loss: 1.1038
Epoch 22/30
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.3335 - loss: 1.1024
Epoch 28/30
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.3333 - loss: 1.1025
Epoch 17/30
 99/144 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3423 - loss: 1.1034

/Users/robin/opt/anaconda3/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.3352 - loss: 1.1057
Epoch 23/30
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.3390 - loss: 1.1039
Epoch 29/30
 18/144 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3511 - loss: 1.1011Epoch 1/30
144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.3281 - loss: 1.1047
129/144 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3547 - loss: 1.1023Epoch 18/30
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.3536 - loss: 1.1026
Epoch 30/30
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.3465 - loss: 1.1042
Epoch 24/30
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.3367 - loss: 1.1035
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.3296 - loss: 1.1024
Epoch 25/30
144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.3280 - loss: 1.1045
Epoch 19/30
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.3322 - loss: 1.1063
Epoch 26/30
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/stepep - accuracy: 0.3186 - loss: 1.

/Users/robin/opt/anaconda3/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.3392 - loss: 1.1025
Epoch 20/30
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.3362 - loss: 1.1066
Epoch 28/30
102/144 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3338 - loss: 1.1122  Epoch 1/30
144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.3344 - loss: 1.1100
Epoch 21/30
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.3485 - loss: 1.1027
Epoch 29/30
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.3211 - loss: 1.1052
Epoch 22/30
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.3367 - loss: 1.1078
Epoch 30/30
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.3340 - loss: 1.1054
Epoch 23/30
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.3284 - loss: 1.1041
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.3410 - loss: 1.1021
Epoch 24/30
144/144 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - accuracy: 0.3361 - loss: 1.4800
Epoch 2/30
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.

...

14/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3372 - loss: 1.0999   Epoch 1/30
38/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3305 - loss: 1.1010

/Users/robin/opt/anaconda3/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.3278 - loss: 1.1010
Epoch 24/30
 6/72 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.3413 - loss: 1.1017 Epoch 1/30
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3372 - loss: 1.1037
Epoch 25/30
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3222 - loss: 1.2379
Epoch 26/30
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3188 - loss: 1.1136
Epoch 27/30
72/72 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.3511 - loss: 1.6406
Epoch 2/30
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3158 - loss: 1.1026
Epoch 28/30
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3441 - loss: 1.1004
Epoch 29/30
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.3562 - loss: 1.5853
Epoch 3/30
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.3340 - loss: 1.1012
Epoch 30/30
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.3476 - loss: 1.4280
Epoch 4/30
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3239 - loss: 1.

/Users/robin/opt/anaconda3/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
108/108 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.5457 - loss: 0.8959
Epoch 2/30
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8897 - loss: 0.3000
Epoch 3/30
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9375 - loss: 0.1778
Epoch 4/30
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9543 - loss: 0.1193
Epoch 5/30
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9592 - loss: 0.1043
Epoch 6/30
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9671 - loss: 0.0930
Epoch 7/30
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9624 - loss: 0.0884
Epoch 8/30
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9636 - loss: 0.0789
Epoch 9/30
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9685 - loss: 0.0690
Epoch 10/30
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9722 - loss: 0.0666
Epoch 11/30
108/108 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9702 - loss: 0.0639
Epoch 12/30
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step

#### Visualise the training and validation loss and accuracy

In [14]:
# Summarize the results from the grid search
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# Extract the mean and standard deviation of test scores, and the corresponding parameters
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

# Print the results in a readable format
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


Best: 0.890385 using {'batch_size': 64, 'epochs': 30, 'layers': 2, 'learning_rate': 0.01, 'nodes': 64}
0.849262 (0.002903) with: {'batch_size': 32, 'epochs': 20, 'layers': 1, 'learning_rate': 0.001, 'nodes': 32}
0.859398 (0.002516) with: {'batch_size': 32, 'epochs': 20, 'layers': 1, 'learning_rate': 0.001, 'nodes': 64}
0.864755 (0.005229) with: {'batch_size': 32, 'epochs': 20, 'layers': 1, 'learning_rate': 0.001, 'nodes': 128}
0.877643 (0.002491) with: {'batch_size': 32, 'epochs': 20, 'layers': 1, 'learning_rate': 0.01, 'nodes': 32}
0.879380 (0.005020) with: {'batch_size': 32, 'epochs': 20, 'layers': 1, 'learning_rate': 0.01, 'nodes': 64}
0.880973 (0.004315) with: {'batch_size': 32, 'epochs': 20, 'layers': 1, 'learning_rate': 0.01, 'nodes': 128}
0.858239 (0.002566) with: {'batch_size': 32, 'epochs': 20, 'layers': 1, 'learning_rate': 0.1, 'nodes': 32}
0.852737 (0.003950) with: {'batch_size': 32, 'epochs': 20, 'layers': 1, 'learning_rate': 0.1, 'nodes': 64}
0.855343 (0.007673) with: {'ba

In [15]:
import matplotlib.pyplot as plt

def plot_training_history(history):
    """
    Plots the training and validation loss and accuracy.

    Args:
        history: Keras History object returned by model.fit().
    """
    # Plot training & validation accuracy values
    plt.figure(figsize=(14, 6))
    
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    # Plot training & validation loss values
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    plt.show()


In [17]:
# Retrieve the best model from GridSearchCV
best_model = grid_result.best_estimator_

# Make predictions on the test set
y_test_pred = best_model.predict(X_test_nouns)

# Evaluate the model on the test set
test_accuracy = best_model.score(X_test_nouns, y_test)
print("Test set accuracy: {:.4f}".format(test_accuracy))


16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Test set accuracy: 0.6409


In [18]:
from sklearn.metrics import classification_report, confusion_matrix

# Generate a classification report
print(classification_report(y_test, y_test_pred))

# Generate a confusion matrix
conf_matrix = confusion_matrix(y_test, y_test_pred)
print(conf_matrix)


              precision    recall  f1-score   support

    negative       0.43      0.43      0.43       121
     neutral       0.75      0.76      0.75       576
    positive       0.50      0.48      0.49       272

    accuracy                           0.64       969
   macro avg       0.56      0.56      0.56       969
weighted avg       0.64      0.64      0.64       969

[[ 52  39  30]
 [ 36 438 102]
 [ 32 109 131]]
